In [1]:
import os
# 원하는 디렉토리로 이동
#os.chdir("/workspace")
# 현재 작업 디렉토리 확인
print("Current working directory:", os.getcwd())

Current working directory: /workspace


In [1]:

import numpy as np
import os
import random
import math
import time
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import glob as gb
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from lightning.pytorch.utilities import CombinedLoader

# (src.model.models_cola) 안의 Cola, ColaMD 모델
from src.model.models_cola import Cola, ColaMD


from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from lightning.pytorch import seed_everything
from sklearn.model_selection import train_test_split

from src.util import random_crop, random_mask, random_multiply, crop_first
from src.model.models_eval import AudioClassifier, AudioClassifierCLAP, AudioClassifierAudioMAE
from src.benchmark.model_util import get_encoder_path, initialize_pretrained_model
from src.util import train_test_split_from_list
import collections
from tqdm import tqdm
from pathlib import Path

torch.backends.cudnn.deterministic = True

In [2]:
def process_mydata_interpatient(data_dir="data/Yeo/", feature_dir="feature/yeo_eval/fixed_split/", split = False):
    """
    오디오 데이터를 inter-patient 방식으로 train/val/test 분할하여
    sound_dir_loc.npy, labels.npy, split.npy 파일을 저장합니다.
    
    [폴더 구조 예시]
    data_dir = "audio/"
      ├── normal/*.wav      (ex: 1.wav, 2.wav, 3.wav, ...)
      └── abnormal/*.wav    (ex: WEBSS-002 T1.wav, WEBSS-002 T2.wav, ...)
    
    [출력]
    feature_dir = "feature/mydata/"
      ├── sound_dir_loc.npy  : 각 파일 경로 (str) 목록
      ├── labels.npy         : 0(정상) or 1(비정상) 라벨
      └── split.npy          : "train" / "val" / "test" 문자열 목록
    """

    os.makedirs(feature_dir, exist_ok=True)

    # 1) 파일 목록 수집
    normal_files = gb.glob(os.path.join(data_dir, "normal", "*.wav"))
    abnormal_files = gb.glob(os.path.join(data_dir, "abnormal", "*.wav"))

    # 2) patient_id 추출 함수
    def get_patient_id(filepath, label_type):
        """
        label_type = 'normal' or 'abnormal'
        filepath 예:
          - normal -> audio/normal/1.wav
          - abnormal -> audio/abnormal/WEBSS-002 T1.wav
        반환: 환자 ID (str)
        """
        basename = os.path.basename(filepath)  # 예: "1.wav" or "WEBSS-002 T1.wav"
        name_only = os.path.splitext(basename)[0]  # 확장자 제거: "1" or "WEBSS-002 T1"
        
        if label_type == "normal":
            # 예: "1" -> "normal_1"
            return f"normal_{name_only}"
        else:
            # abnormal: 예: "WEBSS-002 T1"
            # "WEBSS-002" 부분을 split, 그 중간 "002" 뽑기
            # 실제 파일명이 어떻게 생겼는지에 따라 parsing 로직 조정
            # 아래는 예시 (WEBSS-002를 -로 split -> ["WEBSS", "002 T1"] -> 다시 공백 split -> ["002", "T1"])
            parts = name_only.split('-', maxsplit=1)   # ["WEBSS", "002 T1"]
            if len(parts) == 2:
                second_part = parts[1].strip()         # "002 T1"
                pid = second_part.split()[0]          # "002"
                return pid
            else:
                # 혹시 파일명 구조가 다른 경우 대비
                return name_only

    # 3) 각 파일마다 label(0/1), patient_id를 추출
    filenames = []
    labels = []
    patient_ids = []

    # 정상(0)
    for nf in normal_files:
        filenames.append(nf)
        labels.append(0)
        patient_ids.append(get_patient_id(nf, "normal"))

    # 비정상(1)
    for af in abnormal_files:
        filenames.append(af)
        labels.append(1)
        patient_ids.append(get_patient_id(af, "abnormal"))

    filenames = np.array(filenames)
    labels = np.array(labels)
    patient_ids = np.array(patient_ids)

    print(f"[process_mydata_interpatient] normal:{len(normal_files)}개, abnormal:{len(abnormal_files)}개, 총 {len(filenames)}개 파일")

    # 4) 고유 환자 ID 목록
    unique_pids = np.unique(patient_ids)
    print(f" - 고유 환자 수: {len(unique_pids)}")

    if split == True:
        # 5) 환자 ID 단위로 train/val/test 분할
        #    여기서는 예시로 6:2:2
        train_pids, test_pids = train_test_split(unique_pids, test_size=0.2, random_state=42)
        train_pids, val_pids  = train_test_split(train_pids,  test_size=0.25, random_state=42)
        # => train 60%, val 20%, test 20%
        
        train_pids = set(train_pids)
        val_pids   = set(val_pids)
        test_pids  = set(test_pids)
    
        split_list = []
        for pid in patient_ids:
            if pid in train_pids:
                split_list.append("train")
            elif pid in val_pids:
                split_list.append("val")
            else:
                split_list.append("test")
        split_list = np.array(split_list)
        np.save(os.path.join(feature_dir, "split.npy"), split_list)

    # 6) 저장
    np.save(os.path.join(feature_dir, "sound_dir_loc.npy"), filenames)
    np.save(os.path.join(feature_dir, "labels.npy"), labels)
    np.save(os.path.join(feature_dir, "patient_ids.npy"), patient_ids)
    
    print(" - 저장 완료!")


In [3]:
def extract_and_save_embeddings(feature_dir="feature/yeo_eval/", 
                                pretrain="operaCT", 
                                input_sec=8, 
                                dim=768):
    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
    from src.benchmark.model_util import extract_opera_feature

    opera_features = extract_opera_feature(
        sound_dir_loc, pretrain=pretrain, input_sec=input_sec, dim=dim
    )
    feature_name = pretrain + str(dim)
    np.save(os.path.join(feature_dir, f"{feature_name}_feature.npy"), np.array(opera_features))
    print(f"[extract_and_save_embeddings] {feature_name}_feature.npy 저장 완료!")


In [92]:
data_dir = "data/Yeo/"
feature_dir = "feature/yeo_long/"

pretrain = "operaCT"
dim = 768
min_len_htsat = 60


process_mydata_interpatient(data_dir, feature_dir)

extract_and_save_embeddings(
    feature_dir=feature_dir, 
    pretrain="operaCT",   
    input_sec=min_len_htsat, 
    dim=768
)

[process_mydata_interpatient] normal:19개, abnormal:13개, 총 32개 파일
 - 고유 환자 수: 24
 - 저장 완료!
extracting feature from operaCT model with input_sec 60




                                                   | 0/32 [00:00<?, ?it/s]

▍                                          | 1/32 [00:00<00:17,  1.80it/s]

█▊                                         | 2/32 [00:01<00:14,  2.00it/s]

███▏                                       | 3/32 [00:01<00:13,  2.16it/s]

████▌                                      | 4/32 [00:02<00:14,  1.98it/s]

█████▉                                     | 5/32 [00:02<00:16,  1.62it/s]

███████▎                                   | 6/32 [00:03<00:14,  1.81it/s]

████████▋                                  | 7/32 [00:03<00:13,  1.87it/s]

██████████                                 | 8/32 [00:04<00:12,  1.95it/s]

███████████▍                               | 9/32 [00:04<00:11,  1.99it/s]

████████████▍                             | 10/32 [00:05<00:12,  1.80it/s]

█████████████▊                            | 11/32 [00:06<00:15,  1.40it/s]

███████████████▏                          | 12/32 [00:06<00:12,  1.58it/s]

██████████

(32, 768)
[extract_and_save_embeddings] operaCT768_feature.npy 저장 완료!


In [4]:


class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, data, max_len=256, augment=True, from_npy=False, crop_mode="first", from_audio=False):
        self.data = data[0]
        self.label = data[1]
        self.max_len = max_len
        self.augment = augment
        self.from_npy = from_npy
        self.crop_mode = crop_mode
        self.from_audio = from_audio

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.from_npy:
            npy_path = self.data[idx]
            x = np.load(npy_path + ".npy")
        else:
            x = self.data[idx]

        label = self.label[idx]

        if self.from_audio:
            return x, label

        if self.max_len:
            if self.crop_mode == "random":
                x = random_crop(x, crop_size=self.max_len)
            else:
                x = crop_first(x, crop_size=self.max_len)

        if self.augment:
            x = random_mask(x)
            x = random_multiply(x)

        x = torch.tensor(x, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.long)

        return x, label




class DecayLearningRate(pl.Callback):
    def __init__(self):
        self.old_lrs = []

    def on_train_start(self, trainer, pl_module):
        # track the initial learning rates
        for opt_idx, optimizer in enumerate(trainer.optimizers):
            group = []
            for param_group in optimizer.param_groups:
                group.append(param_group["lr"])
            self.old_lrs.append(group)

    def on_train_epoch_end(self, trainer, pl_module):
        for opt_idx, optimizer in enumerate(trainer.optimizers):
            old_lr_group = self.old_lrs[opt_idx]
            new_lr_group = []
            for p_idx, param_group in enumerate(optimizer.param_groups):
                old_lr = old_lr_group[p_idx]
                new_lr = old_lr * 0.99
                new_lr_group.append(new_lr)
                param_group["lr"] = new_lr
            self.old_lrs[opt_idx] = new_lr_group


In [12]:
def finetune_mydataset(feature_dir="feature/yeo_eval/fixed_split/",
                       pretrain="operaCT",
                       n_cls=2, 
                       l2_strength=1e-4, 
                       epochs=64, 
                       batch_size=512, 
                       lr=1e-4, 
                       head="linear", 
                       feat_dim=768,
                       seed = str(seed),
                      input_sec = 8):
    print("*" * 48)
    print("Training dataset (normal/abnormal) from model pretrained on", pretrain)
    print(f"l2_strength={l2_strength}, lr={lr}, head={head}, feat_dim={feat_dim}")


    # 1) spectrogram 파일이 없다면 생성
    spec_npy = os.path.join(feature_dir, "spectrogram_pad8.npy")
    if not os.path.exists(spec_npy):
        from src.util import get_split_signal_librosa
        sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
        x_data_list = []
        for audio_file in sound_dir_loc:
            # 예: get_split_signal_librosa("", "audio/normal/1", spectrogram=True, input_sec=8, ...)
            # 실제 인자/경로는 사용자 로직에 맞게 수정
            spec_data = get_split_signal_librosa("", audio_file[:-4], 
                                                 spectrogram=True, 
                                                 input_sec=input_sec, 
                                                 trim_tail=False)[0]
            # spec_data.shape이 (T, F) 2차원이라면, 여기서 (1, T, F)로 만들어야
            # batch를 위해 리스트에 append
            x_data_list.append(spec_data)
        x_data = np.array(x_data_list)  # shape: (N, T, F)
        print("x_data.shape:", x_data.shape)
        np.save(spec_npy, x_data)
        print("Saved spectrogram_pad8.npy!")
    else:
        x_data = np.load(spec_npy)  # shape: (N, T, F)

        # 2) 라벨/분할
    splits = np.load(os.path.join(feature_dir, "split.npy"))    # ex) "train"/"val"/"test"
    labels = np.load(os.path.join(feature_dir, "labels.npy"))   # shape: (N,)

    # 3) 모델 초기화
    #    pretrained_model: HTSAT/OperaCT 등 (오디오 입력을 처리할 수 있는 구조)
    pretrained_model = initialize_pretrained_model(pretrain)
    if pretrain != "null":
        encoder_path = get_encoder_path(pretrain)
        print("Loading weights from:", encoder_path)
        ckpt = torch.load(encoder_path)
        pretrained_model.load_state_dict(ckpt["state_dict"], strict=False)
    else:
        print("Training from scratch")

    # AudioClassifier 사용(HTSAT 처럼 4D 입력 처리)
    # 여기서 from_audio=False였던 걸 True로 하거나, 
    # 또는 ICBHI 예제처럼 그냥 2D (T,F)인 걸 (1,T,F)로 만들도록 Dataset에서 조정
    # freeze_encoder 로직은 필요에 따라
    freeze_encoder = "none"
    net = pretrained_model.encoder
    model = AudioClassifier(
        net=net, 
        head=head, 
        classes=n_cls, 
        lr=lr, 
        l2_strength=l2_strength, 
        feat_dim=feat_dim, 
        freeze_encoder=freeze_encoder
    )
    
    # 4) split에 따라 데이터셋 구성
    from torch.utils.data import Dataset, DataLoader
    train_idx = (splits == "train")
    val_idx   = (splits == "val")
    test_idx  = (splits == "test")

    x_train, y_train = x_data[train_idx], labels[train_idx]
    x_val,   y_val   = x_data[val_idx],   labels[val_idx]
    x_test,  y_test  = x_data[test_idx],  labels[test_idx]

    # 5) Dataset / DataLoader
    train_dataset = AudioDataset((x_train, y_train), augment=True, max_len=False)
    val_dataset   = AudioDataset((x_val,   y_val),   augment=False, max_len=False)
    test_dataset  = AudioDataset((x_test,  y_test),  augment=False, max_len=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=2)
    val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=2)

    # 6) Trainer 설정

    logger = CSVLogger(
        save_dir="cks/logs/finetune",
        name="mydata",
        version="_".join([head, pretrain, str(batch_size), str(lr), str(epochs), str(l2_strength)])
    )

    checkpoint_callback = ModelCheckpoint(
        monitor="valid_auc",
        mode="max",
        dirpath="cks/finetune/mydata/", 
        filename="_".join([
            "finetuning", head, pretrain, str(batch_size), str(lr), str(epochs), str(l2_strength)
        ]) + "-{epoch:02d}-{valid_auc:.2f}",
        every_n_epochs=3,
    )

    trainer = pl.Trainer(
        max_epochs=epochs,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1,
        #logger=logger,
        logger=False,
        callbacks=[DecayLearningRate(), checkpoint_callback],
        gradient_clip_val=1.0,
        log_every_n_steps=1,
        enable_progress_bar=True
        
    )

    # 7) 학습
    trainer.fit(model, train_loader, val_loader)

    # 8) 평가
    print("Evaluate on Train set:")
    trainer.test(dataloaders=train_loader)
    print("Evaluate on Val set:")
    trainer.test(dataloaders=val_loader)
    print("Evaluate on Test set:")
    test_res = trainer.test(dataloaders=test_loader)
    if len(test_res) > 0 and "test_auc" in test_res[0]:
        auc = test_res[0]["test_auc"]
        print("Test AUC:", auc)
        acc = test_res[0]["test_acc"]
        print("Test ACC:", acc)
        
        return auc, acc
    else:
        return None

In [6]:
def process_mydata_interpatient_for_loocv(
    data_dir="data/Yeo/", 
    feature_dir="feature/yeo_loocv/"
):
    """
    오디오 데이터를 inter-patient 방식으로 환자 아이디/라벨/파일 목록만 준비해두고,
    LOOCV를 위해 patient_ids.npy, labels.npy, sound_dir_loc.npy, unique_pids.npy를 저장한다.
    """

    import os
    import glob as gb
    import numpy as np

    os.makedirs(feature_dir, exist_ok=True)

    # 1) 파일 목록 수집
    normal_files = gb.glob(os.path.join(data_dir, "normal", "*.wav"))
    abnormal_files = gb.glob(os.path.join(data_dir, "abnormal", "*.wav"))

    # 2) patient_id 추출
    def get_patient_id(filepath, label_type):
        basename = os.path.basename(filepath)  # 예: "1.wav" or "WEBSS-002 T1.wav"
        name_only = os.path.splitext(basename)[0]  # 확장자 제거: "1" or "WEBSS-002 T1"
        
        if label_type == "normal":
            # "1" -> "normal_1"
            return f"normal_{name_only}"
        else:
            parts = name_only.split('-', maxsplit=1)  # ["WEBSS", "002 T1"]
            if len(parts) == 2:
                second_part = parts[1].strip()         # "002 T1"
                pid = second_part.split()[0]          # "002"
                return pid
            else:
                return name_only

    # 3) 각 파일마다 label(0/1), patient_id를 추출
    filenames = []
    labels = []
    patient_ids = []

    # 정상(0)
    for nf in normal_files:
        filenames.append(nf)
        labels.append(0)
        patient_ids.append(get_patient_id(nf, "normal"))

    # 비정상(1)
    for af in abnormal_files:
        filenames.append(af)
        labels.append(1)
        patient_ids.append(get_patient_id(af, "abnormal"))

    filenames = np.array(filenames)
    labels = np.array(labels)
    patient_ids = np.array(patient_ids)

    print(f"[process_mydata_interpatient_for_loocv] normal:{len(normal_files)}개, abnormal:{len(abnormal_files)}개, 총 {len(filenames)}개 파일")

    # 4) 고유 환자 ID 목록
    unique_pids = np.unique(patient_ids)
    print(f" - 고유 환자 수: {len(unique_pids)}")

    # 5) 저장
    np.save(os.path.join(feature_dir, "sound_dir_loc.npy"), filenames)
    np.save(os.path.join(feature_dir, "labels.npy"), labels)
    np.save(os.path.join(feature_dir, "patient_ids.npy"), patient_ids)
    np.save(os.path.join(feature_dir, "unique_pids.npy"), unique_pids)
    
    print(" - 저장 완료!")



In [7]:
def extract_and_save_embeddings_loocv(
    feature_dir="feature/yeo_loocv/", 
    pretrain="operaCT", 
    input_sec=8, 
    dim=768
):
    import os
    import numpy as np

    from src.benchmark.model_util import extract_opera_feature

    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
    opera_features = extract_opera_feature(
        sound_dir_loc, pretrain=pretrain, input_sec=input_sec, dim=dim
    )
    feature_name = pretrain + str(dim)
    np.save(os.path.join(feature_dir, f"{feature_name}_feature.npy"), np.array(opera_features))
    print(f"[extract_and_save_embeddings] {feature_name}_feature.npy 저장 완료!")


In [8]:
import torch.utils.data as data

class AudioDataset(data.Dataset):
    def __init__(self, data, max_len=None, augment=False):
        """
        data: (x_array, label_array)
        x_array: shape (N, T, F) (float)
        label_array: shape (N,) (int)
        
        max_len:  int or None. 크롭 길이
        augment:  True/False
        """
        self.x_data = data[0]
        self.labels = data[1]
        self.max_len = max_len
        self.augment = augment

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]  # (T, F) numpy array
        y = self.labels[idx]

        # 예: 크롭/오그멘테이션 (필요하다면)
        if self.max_len is not None and self.max_len > 0:
            x = crop_first(x, crop_size=self.max_len)  # 사용자 정의 crop 함수
        if self.augment:
            x = random_mask(x)
            x = random_multiply(x)

        # numpy -> torch tensor
        x = torch.tensor(x, dtype=torch.float)
        y = torch.tensor(y, dtype=torch.long)
        return x, y

In [9]:
import os
import numpy as np
import torch
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

# (사용자 정의) 모델/콜백
# from your_model_file import AudioClassifier, DecayLearningRate, initialize_pretrained_model, get_encoder_path
def train_loocv(
    feature_dir="feature/yeo_loocv/",
    input_sec=8.0,
    batch_size=64,
    epochs=10,
    lr=1e-4,
    l2_strength=1e-4,
    head="linear",
    feat_dim=768,
    seed=42
):
    """
    환자 단위 LOOCV 수행 예시.
    (스펙트로그램 = spectrogram_pad{input_sec}.npy 사용)
    """
    import pytorch_lightning as pl
    from torch.utils.data import DataLoader

    pl.seed_everything(seed, workers=True)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    # 1) 데이터 불러오기
    patient_ids = np.load(os.path.join(feature_dir, "patient_ids.npy"))
    labels = np.load(os.path.join(feature_dir, "labels.npy"))
    unique_pids = np.load(os.path.join(feature_dir, "unique_pids.npy"))

    # 2) 스펙트로그램 불러오기 (없으면 생성)
    spec_npy = os.path.join(feature_dir, f"spectrogram_pad{int(input_sec)}.npy")
    if not os.path.exists(spec_npy):
        print(f"[train_loocv] {spec_npy} 파일이 없어 새로 생성합니다...")
        sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
        x_data_list = []
        for audio_file in sound_dir_loc:
            # get_split_signal_librosa:
            # 실제 인자나 로직은 사용자 환경에 맞춰 변경하세요.
            spec_data = get_split_signal_librosa(
                "", 
                audio_file[:-4],  # ".wav" 제거
                spectrogram=True, 
                input_sec=input_sec,  # <= 여기서 input_sec 사용
                trim_tail=False
            )[0]
            x_data_list.append(spec_data)
        x_data = np.array(x_data_list)  # shape: (N, T, F)
        print("x_data.shape:", x_data.shape)
        np.save(spec_npy, x_data)
        print(f"[train_loocv] Saved {spec_npy}!")
    else:
        x_data = np.load(spec_npy)

    print(f"[train_loocv] x_data.shape={x_data.shape}, labels.shape={labels.shape}, unique_pids={len(unique_pids)}")

    # 3) LOOCV 결과 저장할 리스트
    loocv_results = []

    # 4) LOOCV Loop
    for fold_idx, test_pid in enumerate(unique_pids):
        print("\n" + "="*60)
        print(f"[LOOCV] Fold {fold_idx+1}/{len(unique_pids)} | test_pid = {test_pid}")

        # (a) 환자 단위 split
        test_idx = (patient_ids == test_pid)
        train_val_pids = unique_pids[unique_pids != test_pid]

        # (b) train/val 분할 (80:20 예시)
        train_pids, val_pids = train_test_split(
            train_val_pids, test_size=0.2, random_state=seed
        )
        train_idx = np.isin(patient_ids, train_pids)
        val_idx   = np.isin(patient_ids, val_pids)

        x_train, y_train = x_data[train_idx], labels[train_idx]
        x_val,   y_val   = x_data[val_idx],   labels[val_idx]
        x_test,  y_test  = x_data[test_idx],  labels[test_idx]

        print(f"  - train samples = {len(x_train)}, val samples = {len(x_val)}, test samples = {len(x_test)}")

        # (c) Dataset / Dataloader (사용자 정의 AudioDataset 예시)
        train_dataset = AudioDataset((x_train, y_train), augment=True,  max_len=None)
        val_dataset   = AudioDataset((x_val,   y_val),   augment=False, max_len=None)
        test_dataset  = AudioDataset((x_test,  y_test),  augment=False, max_len=None)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
        test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

        # (d) 모델 초기화 (예시)
        #     - OperaCT 프리트레인 로드
        pretrained_model = initialize_pretrained_model("operaCT")
        encoder_path = get_encoder_path("operaCT")
        ckpt = torch.load(encoder_path, map_location="cpu")
        pretrained_model.load_state_dict(ckpt["state_dict"], strict=False)

        # 실제 학습 모델 (AudioClassifier 예시)
        net = pretrained_model.encoder
        model = AudioClassifier(
            net=net, 
            head=head, 
            classes=2, 
            lr=lr, 
            l2_strength=l2_strength, 
            feat_dim=feat_dim, 
            freeze_encoder="none"  
        )

        # (e) Trainer 설정
        trainer = pl.Trainer(
            max_epochs=epochs,
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            devices=1,
            logger=False,   # 필요하다면 CSVLogger 사용
            callbacks=[DecayLearningRate(), 
                       ModelCheckpoint(
                           monitor="valid_auc",
                           mode="max",
                           dirpath="cks/finetune/mydata/loocv/", 
                           filename=(
                               f"finetuning_{head}_operaCT_{batch_size}_{lr}_{epochs}_{l2_strength}"
                               "-{epoch:02d}-{valid_auc:.2f}"
                           ),
                           every_n_epochs=3,
                       )
                      ],
            gradient_clip_val=1.0,
            log_every_n_steps=1,
            enable_progress_bar=True
        )

        # (f) 학습
        trainer.fit(model, train_loader, val_loader)

        # (g) 테스트
        test_result = trainer.test(model, test_loader)
        if len(test_result) > 0 and "test_auc" in test_result[0]:
            test_auc = test_result[0]["test_auc"]
            # 어떤 환자가 성능이 안 나오는지 확인하기 위해 PID + AUC 출력
            print(f"[LOOCV fold={fold_idx+1}] test_pid = {test_pid}, Test AUC = {test_auc:.4f}")
            loocv_results.append(test_auc)
        else:
            print(f"[LOOCV fold={fold_idx+1}] test_pid = {test_pid}, Test AUC 없음 ...")
            loocv_results.append(np.nan)

    # 5) 결과 요약
    loocv_results = np.array(loocv_results, dtype=np.float32)
    valid_results = loocv_results[~np.isnan(loocv_results)]
    mean_auc = np.mean(valid_results) if len(valid_results) > 0 else 0.0
    std_auc  = np.std(valid_results) if len(valid_results) > 1 else 0.0

    print("="*60)
    print(f"[LOOCV] 전체 fold 개수 = {len(unique_pids)}")
    print("각 fold Test AUC:", loocv_results)
    print(f"Mean Test AUC: {mean_auc:.4f} ± {std_auc:.4f}")
    print("="*60)

    return mean_auc, std_auc, loocv_results


In [104]:
if __name__ == "__main__":
    # 1) 환자 id/라벨/파일목록 전처리
    process_mydata_interpatient_for_loocv(
        data_dir="data/Yeo/",
        feature_dir="feature/yeo_loocv_long/"
    )

    # 2) (선택) OperaCT 임베딩을 미리 추출해 저장
    extract_and_save_embeddings_loocv(
        feature_dir="feature/yeo_loocv_long/",
        pretrain="operaCT",
        input_sec=60,
        dim=768
    )

    # 3) LOOCV 학습
    mean_auc, std_auc, all_fold_auc = train_loocv(
        feature_dir="feature/yeo_loocv_long/",
        input_sec=60,
        batch_size=512,
        epochs=64,
        lr=1e-4,
        l2_strength=1e-4,
        head="linear",
        feat_dim=768,
        seed=42
    )

    print("LOOCV 최종 결과:", mean_auc, std_auc, all_fold_auc)

[process_mydata_interpatient_for_loocv] normal:19개, abnormal:13개, 총 32개 파일
 - 고유 환자 수: 24
 - 저장 완료!
extracting feature from operaCT model with input_sec 60




                                                   | 0/32 [00:00<?, ?it/s]

▍                                          | 1/32 [00:00<00:13,  2.22it/s]

█▊                                         | 2/32 [00:00<00:13,  2.30it/s]

███▏                                       | 3/32 [00:01<00:10,  2.64it/s]

████▌                                      | 4/32 [00:01<00:10,  2.80it/s]

█████▉                                     | 5/32 [00:02<00:16,  1.66it/s]

███████▎                                   | 6/32 [00:03<00:14,  1.80it/s]

████████▋                                  | 7/32 [00:03<00:13,  1.91it/s]

██████████                                 | 8/32 [00:07<00:36,  1.52s/it]

███████████▍                               | 9/32 [00:08<00:33,  1.45s/it]

████████████▍                             | 10/32 [00:08<00:25,  1.15s/it]

█████████████▊                            | 11/32 [00:09<00:22,  1.05s/it]

███████████████▏                          | 12/32 [00:10<00:22,  1.11s/it]

██████████

(32, 768)
[extract_and_save_embeddings] operaCT768_feature.npy 저장 완료!
[train_loocv] feature/yeo_loocv_long/spectrogram_pad60.npy 파일이 없어 새로 생성합니다...
x_data.shape: (32, 1876, 64)
[train_loocv] Saved feature/yeo_loocv_long/spectrogram_pad60.npy!
[train_loocv] x_data.shape=(32, 1876, 64), labels.shape=(32,), unique_pids=24

[LOOCV] Fold 1/24 | test_pid = 002
  - train samples = 23, val samples = 8, test samples = 1


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

  | Name | Type             | Params
------------------------------------------
0 | net  | EncoderHTSAT     | 31.3 M
1 | head | Sequential       | 1.5 K 
2 | loss | CrossEntropyLoss | 0     
------------------------------------------
30.2 M    Trainable params
1.1 M     Non-trainable params
31.3 M    Total params
125.307   Total estimated model params size (MB)


768 2


...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=64` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                | 0/? [00:00<?, ?it/s]

test_auc tensor(0.)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_auc                    0.0
        test_loss         0.00010394509445177391
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[LOOCV fold=24] test_pid = normal_9, Test AUC = 0.0000
[LOOCV] 전체 fold 개수 = 24
각 fold Test AUC: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Mean Test AUC: 0.0000 ± 0.0000
LOOCV 최종 결과: 0.0 0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Fix Split But Stratified

In [10]:
import os
import glob as gb
import numpy as np

from sklearn.model_selection import train_test_split

def process_mydata_interpatient_stratified(
    data_dir="data/Yeo/", 
    feature_dir="feature/yeo_strat_split/", 
    do_split=True,
    test_size=0.2,
    val_size=0.2,
    random_state=42
):
    """
    오디오 데이터를 (inter-patient + stratified) 방식으로 train/val/test 분할하여
    sound_dir_loc.npy, labels.npy, patient_ids.npy, split.npy 파일을 저장.

    [폴더 구조 예시]
    data_dir = "audio/"
      ├── normal/*.wav
      └── abnormal/*.wav

    [출력]
    feature_dir = "feature/mydata/"
      ├── sound_dir_loc.npy  : 각 파일 경로 (str) 목록
      ├── labels.npy         : 0(정상) or 1(비정상) 라벨
      ├── patient_ids.npy    : 환자 ID (str)
      └── split.npy          : "train"/"val"/"test" 문자열 목록
    """

    os.makedirs(feature_dir, exist_ok=True)

    # 1) 파일 목록 수집
    normal_files = gb.glob(os.path.join(data_dir, "normal", "*.wav"))
    abnormal_files = gb.glob(os.path.join(data_dir, "abnormal", "*.wav"))

    # 2) patient_id 추출 함수
    def get_patient_id(filepath, label_type):
        """
        label_type = 'normal' or 'abnormal'
        filepath 예:
          - normal -> audio/normal/1.wav
          - abnormal -> audio/abnormal/WEBSS-002 T1.wav
        반환: 환자 ID (str)
        """
        basename = os.path.basename(filepath)  # 예: "1.wav" or "WEBSS-002 T1.wav"
        name_only = os.path.splitext(basename)[0]  # 확장자 제거: "1" or "WEBSS-002 T1"
        
        if label_type == "normal":
            # 예: "1" -> "normal_1"
            return f"normal_{name_only}"
        else:
            # abnormal: 예: "WEBSS-002 T1"
            parts = name_only.split('-', maxsplit=1)   # ["WEBSS", "002 T1"]
            if len(parts) == 2:
                second_part = parts[1].strip()         # "002 T1"
                pid = second_part.split()[0]          # "002"
                return pid
            else:
                # 혹시 파일명 구조가 다른 경우 대비
                return name_only

    # 3) 각 파일마다 label(0/1), patient_id를 추출
    filenames = []
    labels = []
    patient_ids = []

    # 정상(0)
    for nf in normal_files:
        filenames.append(nf)
        labels.append(0)
        patient_ids.append(get_patient_id(nf, "normal"))

    # 비정상(1)
    for af in abnormal_files:
        filenames.append(af)
        labels.append(1)
        patient_ids.append(get_patient_id(af, "abnormal"))

    filenames = np.array(filenames)
    labels = np.array(labels)
    patient_ids = np.array(patient_ids)

    print(f"[process_mydata_interpatient_stratified] normal:{len(normal_files)}개, abnormal:{len(abnormal_files)}개, 총 {len(filenames)}개 파일")

    # 4) 고유 환자 ID 목록
    unique_pids = np.unique(patient_ids)
    print(f" - 고유 환자 수: {len(unique_pids)}")

    # 5) (중요) '환자별 라벨'을 만든다.
    #    - 예시: 환자에게 속한 파일 중 하나라도 비정상이면 1, 전부 정상이어야 0
    #    - 환자별로 라벨 취합하여 dict 형태로 만들어두고, 나중에 stratify에 활용
    pid_to_label = {}
    for pid in unique_pids:
        # 해당 pid에 해당하는 파일들의 라벨
        pid_labels = labels[patient_ids == pid]
        if np.any(pid_labels == 1):
            # 하나라도 1이면 비정상
            pid_to_label[pid] = 1
        else:
            pid_to_label[pid] = 0

    # 환자별 라벨 배열(0/1)
    pid_labels_array = np.array([pid_to_label[p] for p in unique_pids])

    # 6) stratified split 수행
    #    test_size=0.2 => test 20%, 남은 train+val 80%
    #    val_size=0.2  => (train+val 중) val 20%, 즉 전체 16%, 최종적으로 train:64%, val:16%, test:20%

    if do_split:
        # (1) train+val / test
        trainval_pids, test_pids = train_test_split(
            unique_pids,
            test_size=test_size,
            random_state=random_state,
            stratify=pid_labels_array  # <-- 여기서 라벨 비율 유지
        )

        # (2) train / val
        #   trainval_pids 중에서 라벨 배열 다시 뽑기
        trainval_labels = np.array([pid_to_label[p] for p in trainval_pids])
        train_pids, val_pids = train_test_split(
            trainval_pids,
            test_size=val_size,
            random_state=random_state,
            stratify=trainval_labels
        )

        train_pids = set(train_pids)
        val_pids   = set(val_pids)
        test_pids  = set(test_pids)

        # 모든 파일에 대해서 split 결정
        split_list = []
        for pid in patient_ids:
            if pid in train_pids:
                split_list.append("train")
            elif pid in val_pids:
                split_list.append("val")
            elif pid in test_pids:
                split_list.append("test")
            else:
                # 혹시 분할에서 누락된 경우(거의 없겠지만)
                split_list.append("etc")
        split_list = np.array(split_list)
    else:
        # split 안 하고, 일괄 "train" 등으로 지정하거나 원하는대로...
        split_list = np.array(["train"] * len(patient_ids))

    # 7) 저장
    np.save(os.path.join(feature_dir, "sound_dir_loc.npy"), filenames)
    np.save(os.path.join(feature_dir, "labels.npy"), labels)
    np.save(os.path.join(feature_dir, "patient_ids.npy"), patient_ids)
    np.save(os.path.join(feature_dir, "split.npy"), split_list)
    
    print(" - 저장 완료!")
    print("분할 결과:", 
          f"train={np.sum(split_list=='train')}, "
          f"val={np.sum(split_list=='val')}, "
          f"test={np.sum(split_list=='test')}")


#####################################
#  이후엔 feature 추출, 모델 학습 시에
#  이 split.npy를 그대로 로드하여
#  train/val/test를 구성하면 됩니다.
#####################################


if __name__ == "__main__":
    data_dir = "data/Yeo/"
    feature_dir = "feature/yeo_strat_long_split/"

    # 예시 실행
    process_mydata_interpatient_stratified(
        data_dir=data_dir, 
        feature_dir=feature_dir, 
        do_split=True,
        test_size=0.2, 
        val_size=0.2, 
        random_state=42
    )



[process_mydata_interpatient_stratified] normal:19개, abnormal:13개, 총 32개 파일
 - 고유 환자 수: 24
 - 저장 완료!
분할 결과: train=17, val=7, test=8


In [13]:

feature_dir="feature/yeo_strat_long_split/"
n_run = 5

auc_scores = []
acc_scores = []
for seed in range(n_run):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    seed_everything(seed, workers=True)
    auc,acc = finetune_mydataset(
        feature_dir=feature_dir,
        pretrain="operaCT",
        n_cls=2, 
        l2_strength=1e-4, 
        epochs=100,       
        batch_size=512,   
        lr=1e-4, 
        head="linear", 
        feat_dim=768,
        seed = str(seed),
        input_sec = 8
    )
    auc_scores.append(auc)
    acc_scores.append(acc)



Seed set to 0


************************************************
Training dataset (normal/abnormal) from model pretrained on operaCT
l2_strength=0.0001, lr=0.0001, head=linear, feat_dim=768


/opt/conda/envs/audio/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loading weights from: cks/model/combined/icbhi_icbhicycle_hf_lung_kauh_PulmonarySound_SPRSound/encoder-experiment_wo Coswara_kauh_Pul_20_SPR_20-epoch=129--valid_acc=0.77-valid_loss=0.8953.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /workspace/cks/finetune/mydata exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

  | Name | Type             | Params
------------------------------------------
0 | net  | EncoderHTSAT     | 31.3 M
1 | head | Sequential       | 1.5 K 
2 | loss | CrossEntropyLoss | 0     
------------------------------------------
30.2 M    Trainable p

...

Evaluate on Test set:


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
Loaded model weights from the checkpoint at /workspace/cks/finetune/mydata/finetuning_linear_operaCT_512_0.0001_100_0.0001-epoch=02-valid_auc=1.00-v20.ckpt


Testing: |                                                | 0/? [00:00<?, ?it/s]

test_auc tensor(0.8125)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                   0.75
        test_auc                  0.8125
        test_loss           0.5866010189056396
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test AUC: 0.8125
Test ACC: 0.75
[0.875, 0.9375, 0.9375, 0.8125, 0.8125]


NameError: name 'pretrain' is not defined

In [14]:
task =  'yeo_data_long_strat'
pretrain = 'OPERACT'
print("=" * 48)
print(auc_scores)
print("Five times mean task {} finetuning from {} results: auc mean {:.3f} ± {:.3f}".format(task, pretrain, np.mean(auc_scores), np.std(auc_scores)) )
print(acc_scores)
print("Five times mean task {} finetuning from {} results: acc mean {:.3f} ± {:.3f}".format(task, pretrain, np.mean(acc_scores), np.std(acc_scores)) )
print("=" * 48)

[0.875, 0.9375, 0.9375, 0.8125, 0.8125]
Five times mean task yeo_data_long_strat finetuning from OPERACT results: auc mean 0.875 ± 0.056
[0.75, 0.875, 0.875, 0.875, 0.75]
Five times mean task yeo_data_long_strat finetuning from OPERACT results: acc mean 0.825 ± 0.061


In [89]:

def check_stratified_distribution(feature_dir):
    # 1) 저장된 파일 로드
    split_list = np.load(os.path.join(feature_dir, "split.npy"))        # e.g. ["train", "train", "test", ...]
    patient_ids = np.load(os.path.join(feature_dir, "patient_ids.npy")) # e.g. ["normal_1", "normal_1", "002", ...]
    labels = np.load(os.path.join(feature_dir, "labels.npy"))          # e.g. [0, 0, 1,  ...]

    unique_pids = np.unique(patient_ids)
    
    # 2) (환자단위) 라벨: pid_to_label
    #    - 파일이 여러 개여도 하나라도 1(비정상)이면 그 환자는 1로 봄
    pid_to_label = {}
    for pid in unique_pids:
        pid_label = labels[patient_ids == pid]
        if np.any(pid_label == 1):
            pid_to_label[pid] = 1
        else:
            pid_to_label[pid] = 0

    # 3) (환자단위) split: pid_to_split
    #    - 한 환자에 속한 파일들은 모두 같은 split에 들어가 있다고 가정(inter-patient split)
    pid_to_split = {}
    for i, pid in enumerate(patient_ids):
        current_split = split_list[i]
        # 한 번도 등록 안 됐으면 등록
        if pid not in pid_to_split:
            pid_to_split[pid] = current_split
        else:
            # 혹시라도 한 환자가 여러 split에 걸쳐 있으면 경고
            if pid_to_split[pid] != current_split:
                print(f"Warning: {pid}가 여러 split에 속해있음! {pid_to_split[pid]} vs {current_split}")

    # 4) split별 환자 목록
    split_names = ["train", "val", "test"]
    split_pid_dict = {s: [] for s in split_names}
    for pid in unique_pids:
        sp = pid_to_split[pid]
        if sp in split_pid_dict:
            split_pid_dict[sp].append(pid)
        # 혹시 "etc" 등 다른 split이 있을 수 있으니 예외 처리할 수도 있음

    # 5) (환자단위) split별 정상/비정상 환자 수
    def count_patients(pid_list):
        n_normal = sum(pid_to_label[pid] == 0 for pid in pid_list)
        n_abnormal = sum(pid_to_label[pid] == 1 for pid in pid_list)
        return n_normal, n_abnormal

    patient_counts = {}
    for sp in split_names:
        pids = split_pid_dict[sp]
        n_normal, n_abnormal = count_patients(pids)
        patient_counts[sp] = (n_normal, n_abnormal, len(pids))

    # 6) (샘플단위) split별 정상/비정상 샘플 수
    #    split_list[i], labels[i]를 직접 카운트하면 됨
    sample_counts = {s: [0, 0, 0] for s in split_names}  # [normal_count, abnormal_count, total]
    for i in range(len(labels)):
        sp = split_list[i]   # "train"/"val"/"test"
        lab = labels[i]      # 0 or 1
        if sp in sample_counts:
            if lab == 0:
                sample_counts[sp][0] += 1
            else:
                sample_counts[sp][1] += 1
            sample_counts[sp][2] += 1

    # 7) 결과 출력
    print("===== Split별 환자 수 (Patient-level) =====")
    for sp in split_names:
        n_normal, n_abnormal, n_total = patient_counts[sp]
        print(f"{sp.upper()}: 환자 normal={n_normal}, abnormal={n_abnormal}, total={n_total}")

    print("\n===== Split별 샘플 수 (File-level) =====")
    for sp in split_names:
        normal_count, abnormal_count, total_count = sample_counts[sp]
        print(f"{sp.upper()}: 파일 normal={normal_count}, abnormal={abnormal_count}, total={total_count}")


if __name__ == "__main__":
    feature_dir = "feature/yeo_eval/fixed_split/"
    check_stratified_distribution(feature_dir)

    print("\nstrat")

    feature_dir = "feature/yeo_strat_split"
    check_stratified_distribution(feature_dir)

    


===== Split별 환자 수 (Patient-level) =====
TRAIN: 환자 normal=12, abnormal=2, total=14
VAL: 환자 normal=3, abnormal=2, total=5
TEST: 환자 normal=4, abnormal=1, total=5

===== Split별 샘플 수 (File-level) =====
TRAIN: 파일 normal=12, abnormal=6, total=18
VAL: 파일 normal=3, abnormal=6, total=9
TEST: 파일 normal=4, abnormal=1, total=5

strat
===== Split별 환자 수 (Patient-level) =====
TRAIN: 환자 normal=12, abnormal=3, total=15
VAL: 환자 normal=3, abnormal=1, total=4
TEST: 환자 normal=4, abnormal=1, total=5

===== Split별 샘플 수 (File-level) =====
TRAIN: 파일 normal=12, abnormal=5, total=17
VAL: 파일 normal=3, abnormal=4, total=7
TEST: 파일 normal=4, abnormal=4, total=8


### Stratified & K fold

In [11]:
import os
import numpy as np
import torch
import pytorch_lightning as pl
from sklearn.model_selection import StratifiedKFold, train_test_split

#############################################################
# (A) k-Fold 교차검증을 위한 함수
#############################################################
def kfold_mydataset(
    feature_dir="feature/yeo_strat_kfold/",
    pretrain="operaCT",
    n_cls=2, 
    l2_strength=1e-4, 
    epochs=64, 
    batch_size=512, 
    lr=1e-4, 
    head="linear", 
    feat_dim=768,
    n_splits=5,            # k-Fold 개수
    val_size=0.2,          # (train fold 중) val 비율
    random_state=42,
    input_sec=8
):
    """
    - k-Fold 교차검증을 수행하여 모든 환자가 'test set'에 한 번씩 포함되도록 함.
    - fold마다 학습 & 테스트 -> 최종 AUC/ACC 평균(표준편차) 산출

    (전제) feature_dir에 이미 다음 파일들이 존재한다고 가정:
      - sound_dir_loc.npy
      - labels.npy
      - patient_ids.npy
    (스펙트로그램이 없다면 함수 내에서 자동 생성 가능)
    """

    print("*" * 48)
    print(f"[kfold_mydataset] n_splits={n_splits}, pretrain={pretrain}, head={head}, input_sec={input_sec}")
    print(f"l2_strength={l2_strength}, lr={lr}, feat_dim={feat_dim}, epochs={epochs}")

    # 0) 시드 고정 (kfold 자체도 난수분할이 있을 수 있음)
    pl.seed_everything(random_state, workers=True)
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed(random_state)

    # 1) 필요한 파일 로드
    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
    labels = np.load(os.path.join(feature_dir, "labels.npy"))
    patient_ids = np.load(os.path.join(feature_dir, "patient_ids.npy"))

    # 2) 스펙트로그램 파일 준비
    spec_npy = os.path.join(feature_dir, f"spectrogram_pad{int(input_sec)}.npy")
    if not os.path.exists(spec_npy):
        from src.util import get_split_signal_librosa
        print(f"[kfold_mydataset] {spec_npy} 파일이 없어 새로 생성합니다...")
        x_data_list = []
        for audio_file in sound_dir_loc:
            spec_data = get_split_signal_librosa(
                "",
                audio_file[:-4],  # ".wav" 제거
                spectrogram=True, 
                input_sec=input_sec,
                trim_tail=False
            )[0]
            x_data_list.append(spec_data)
        x_data = np.array(x_data_list)
        np.save(spec_npy, x_data)
        print(f"[kfold_mydataset] Saved {spec_npy}!")
    else:
        x_data = np.load(spec_npy)

    print(f"Total samples = {len(x_data)}, x_data.shape={x_data.shape}, labels.shape={labels.shape}")

    # 3) '환자별' 라벨 결정 (환자에 속한 파일 중 하나라도 비정상이면 비정상)
    unique_pids = np.unique(patient_ids)
    pid_to_label = {}
    for pid in unique_pids:
        pid_labels = labels[patient_ids == pid]
        if np.any(pid_labels == 1):
            pid_to_label[pid] = 1
        else:
            pid_to_label[pid] = 0
    pid_labels_array = np.array([pid_to_label[p] for p in unique_pids])

    # 4) k-Fold 정의 (StratifiedKFold를 환자 단위로)
    skf = StratifiedKFold(
        n_splits=n_splits, 
        shuffle=True, 
        random_state=random_state
    )

    # 5) fold별 결과 저장 리스트
    fold_auc_list = []
    fold_acc_list = []

    # 6) Fold Loop
    for fold_idx, (trainval_index, test_index) in enumerate(skf.split(unique_pids, pid_labels_array)):
        # test용 환자
        test_pids = unique_pids[test_index]
        # train+val용 환자
        trainval_pids = unique_pids[trainval_index]

        # train+val 환자 라벨
        trainval_labels = np.array([pid_to_label[p] for p in trainval_pids])

        # (a) 다시 train/val 분할
        #     val_size=0.2 => (train+val 80% 중 20% = 16% 전체)
        tr_pids, val_pids = train_test_split(
            trainval_pids,
            test_size=val_size,
            random_state=random_state,
            stratify=trainval_labels
        )

        # (b) 환자 -> index
        tr_idx = np.isin(patient_ids, tr_pids)
        val_idx = np.isin(patient_ids, val_pids)
        test_idx = np.isin(patient_ids, test_pids)

        x_train, y_train = x_data[tr_idx], labels[tr_idx]
        x_val,   y_val   = x_data[val_idx], labels[val_idx]
        x_test,  y_test  = x_data[test_idx], labels[test_idx]

        print("\n" + "="*60)
        print(f"[Fold {fold_idx+1}/{n_splits}] train={len(x_train)}, val={len(x_val)}, test={len(x_test)}")
        print(f"train_id : {tr_pids}, val_id : {val_pids}, test_id: {test_pids},")

        # (c) Dataset / Dataloader (사용자 정의)
        train_dataset = AudioDataset((x_train, y_train), augment=True,  max_len=False)
        val_dataset   = AudioDataset((x_val,   y_val),   augment=False, max_len=False)
        test_dataset  = AudioDataset((x_test,  y_test),  augment=False, max_len=False)

        from torch.utils.data import DataLoader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
        test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

        # (d) 모델 초기화
        pretrained_model = initialize_pretrained_model(pretrain)
        if pretrain != "null":
            encoder_path = get_encoder_path(pretrain)
            ckpt = torch.load(encoder_path, map_location="cpu")
            pretrained_model.load_state_dict(ckpt["state_dict"], strict=False)

        net = pretrained_model.encoder
        model = AudioClassifier(
            net=net, 
            head=head, 
            classes=n_cls, 
            lr=lr, 
            l2_strength=l2_strength, 
            feat_dim=feat_dim, 
            freeze_encoder="none"
        )

        # (e) Trainer 설정
        #     (val_loss 기준이 아닌 valid_auc 기준 체크포인트 예시)
        checkpoint_callback = ModelCheckpoint(
            monitor="valid_auc",
            mode="max",
            dirpath="cks/finetune/kfold_results_60sec/", 
            filename=(
                f"kfold_{fold_idx+1}-{head}_{pretrain}_bs{batch_size}_lr{lr}_ep{epochs}_l2{l2_strength}"
                "-{epoch:02d}-{valid_auc:.2f}"
            ),
            every_n_epochs=5,
        )

        trainer = pl.Trainer(
            max_epochs=epochs,
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            devices=1,
            logger=False,  # 필요 시 CSVLogger
            callbacks=[DecayLearningRate(), checkpoint_callback],
            gradient_clip_val=1.0,
            log_every_n_steps=1,
            enable_progress_bar=True
        )

        # (f) 학습
        trainer.fit(model, train_loader, val_loader)

        # (g) 테스트
        test_result = trainer.test(model, test_loader)
        if len(test_result) > 0:
            if "test_auc" in test_result[0]:
                fold_auc = test_result[0]["test_auc"]
            else:
                fold_auc = np.nan
            if "test_acc" in test_result[0]:
                fold_acc = test_result[0]["test_acc"]
            else:
                fold_acc = np.nan

            print(f"[Fold {fold_idx+1}] Test AUC={fold_auc:.4f}, ACC={fold_acc:.4f}")
            fold_auc_list.append(fold_auc)
            fold_acc_list.append(fold_acc)
        else:
            fold_auc_list.append(np.nan)
            fold_acc_list.append(np.nan)

    # 7) k-Fold 결과 요약
    fold_auc_list = np.array(fold_auc_list, dtype=np.float32)
    fold_acc_list = np.array(fold_acc_list, dtype=np.float32)

    valid_auc = fold_auc_list[~np.isnan(fold_auc_list)]
    valid_acc = fold_acc_list[~np.isnan(fold_acc_list)]

    mean_auc = np.mean(valid_auc) if len(valid_auc) > 0 else 0.0
    std_auc  = np.std(valid_auc)  if len(valid_auc) > 1 else 0.0
    mean_acc = np.mean(valid_acc) if len(valid_acc) > 0 else 0.0
    std_acc  = np.std(valid_acc)  if len(valid_acc) > 1 else 0.0

    print("="*60)
    print(f"Fold AUCs: {fold_auc_list}")
    print(f"Fold ACCs: {fold_acc_list}")
    print(f"Mean AUC: {mean_auc:.4f} ± {std_auc:.4f}")
    print(f"Mean ACC: {mean_acc:.4f} ± {std_acc:.4f}")
    print("="*60)

    return (mean_auc, std_auc, fold_auc_list), (mean_acc, std_acc, fold_acc_list)

In [109]:



#############################################################
# (B) 실행 예시
#############################################################
if __name__ == "__main__":
    # feature_dir 안에 사전에 다음 파일이 준비되어 있어야 합니다:
    #   sound_dir_loc.npy, labels.npy, patient_ids.npy
    #   (split.npy는 필요없습니다. 왜냐하면 k-Fold를 새로 만들기 때문)
    # 만약 spectrogram_pad{input_sec}.npy도 없다면 코드가 자동 생성합니다.
    
    feature_dir = "feature/yeo_strat_kfold_long/"  # 기존에 정리된 데이터 경로
    # 아래 함수는 필요에 따라 기존의 "process_mydata_interpatient_stratified" 대신,
    # 환자 id / 라벨 / 파일목록만 준비하는 로직을 써서
    # (split.npy는 만들지 않아도 됩니다.)

    mean_auc_info, mean_acc_info = kfold_mydataset(
        feature_dir=feature_dir,
        pretrain="operaCT",
        n_cls=2, 
        l2_strength=1e-4, 
        epochs=100,         # 예: 50 epoch
        batch_size=256,    # 예: 256
        lr=1e-4, 
        head="linear", 
        feat_dim=768,
        n_splits=5,        # 5-Fold
        val_size=0.2,      # train 중 20%를 val로 씀
        random_state=42,   # 고정 seed
        input_sec=60       # 8초 스펙트로그램
    )

    # mean_auc_info / mean_acc_info = (평균, 표준편차, 각 fold 결과 array)
    (mean_auc, std_auc, fold_auc_arr) = mean_auc_info
    (mean_acc, std_acc, fold_acc_arr) = mean_acc_info

    print("[Final Result]")
    print("Fold AUCs:", fold_auc_arr)
    print(f"Mean AUC: {mean_auc:.3f} ± {std_auc:.3f}")
    print("Fold ACCs:", fold_acc_arr)
    print(f"Mean ACC: {mean_acc:.3f} ± {std_acc:.3f}")


Seed set to 42


************************************************
[kfold_mydataset] n_splits=5, pretrain=operaCT, head=linear, input_sec=60
l2_strength=0.0001, lr=0.0001, feat_dim=768, epochs=100
[kfold_mydataset] feature/yeo_strat_kfold_long/spectrogram_pad60.npy 파일이 없어 새로 생성합니다...
[kfold_mydataset] Saved feature/yeo_strat_kfold_long/spectrogram_pad60.npy!
Total samples = 32, x_data.shape=(32, 1876, 64), labels.shape=(32,)

[Fold 1/5] train=19, val=5, test=8
train_id : ['normal_19' 'normal_1' '002' 'normal_12' 'normal_9' 'normal_14'
 'normal_8' 'normal_4' '005' '003' 'normal_2' 'normal_5' 'normal_10'
 'normal_6' 'normal_16'], val_id : ['normal_13' 'normal_7' '007' 'normal_17'], test_id: ['006' 'normal_11' 'normal_15' 'normal_18' 'normal_3'],


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

  | Name | Type             | Params
------------------------------------------
0 | net  | EncoderHTSAT     | 31.3 M
1 | head | Sequential       | 1.5 K 
2 | loss | CrossEntropyLoss | 0     
------------------------------------------
30.2 M    Trainable params
1.1 M     Non-trainable params
31.3 M    Total params
125.307   Total estimated model params size (MB)


768 2


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing: |                                                | 0/? [00:00<?, ?it/s]

test_auc tensor(0.7500)
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7142857142857142
        test_auc                   0.75
        test_loss            2.639827251434326
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[Fold 5] Test AUC=0.7500, ACC=0.7143
Fold AUCs: [0.75 1.   1.   1.   0.75]
Fold ACCs: [0.5        1.         0.8333333  1.         0.71428573]
Mean AUC: 0.9000 ± 0.1225
Mean ACC: 0.8095 ± 0.1887
[Final Result]
Fold AUCs: [0.75 1.   1.   1.   0.75]
Mean AUC: 0.900 ± 0.122
Fold ACCs: [0.5        1.         0.8333333  1.         0.71428573]
Mean ACC: 0.810 ± 0.189


In [12]:
import os
import numpy as np
from tqdm import tqdm

def extract_and_save_embeddings_baselines(feature_dir, feature="opensmile"):
    """
    사용 예:
      extract_and_save_embeddings_baselines(feature_dir="feature/yeo_strat_kfold_long/", feature="opensmile")

    인자로 받은 feature에 따라, sound_dir_loc.npy 내의 모든 파일에 대해
    baseline feature를 추출한 뒤 .npy 파일로 저장합니다.
    """
    from src.benchmark.baseline.extract_feature import (
        extract_opensmile_features,
        extract_vgg_feature,
        extract_clap_feature,
        extract_audioMAE_feature
    )

    # 1) 오디오 파일 경로 목록 로드
    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))

    # 2) feature 종류에 맞게 임베딩 추출 -> 저장
    if feature == "opensmile":
        opensmile_features = []
        for file in tqdm(sound_dir_loc, desc="Extracting OpenSmile..."):
            opensmile_feature = extract_opensmile_features(file)
            opensmile_features.append(opensmile_feature)
        np.save(os.path.join(feature_dir, "opensmile_feature.npy"), np.array(opensmile_features))
        print("[Done] Saved opensmile_feature.npy")

    elif feature == "vggish":
        vgg_features = extract_vgg_feature(sound_dir_loc)
        np.save(os.path.join(feature_dir, "vggish_feature.npy"), np.array(vgg_features))
        print("[Done] Saved vggish_feature.npy")

    elif feature == "clap":
        clap_features = extract_clap_feature(sound_dir_loc)
        np.save(os.path.join(feature_dir, "clap_feature.npy"), np.array(clap_features))
        print("[Done] Saved clap_feature.npy")

    elif feature == "audiomae":
        audiomae_feature = extract_audioMAE_feature(sound_dir_loc)
        np.save(os.path.join(feature_dir, "audiomae_feature.npy"), np.array(audiomae_feature))
        print("[Done] Saved audiomae_feature.npy")

    else:
        raise ValueError(f"Unknown feature type: {feature}")


In [13]:
import torch
from torch.utils.data import Dataset

class EmbeddingDataset(Dataset):
    def __init__(self, x_data, y_data):
        """
        x_data: shape = (N, D) 
        y_data: shape = (N,)
        """
        super().__init__()
        self.x_data = torch.tensor(x_data, dtype=torch.float)
        self.y_data = torch.tensor(y_data, dtype=torch.long)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]



In [14]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torchmetrics import AUROC, Accuracy

class EmbeddingClassifier(pl.LightningModule):
    def __init__(
        self,
        input_dim,       # 임베딩 차원 (예: opensmile 등에서 얻은 feature dimension)
        n_cls=2,
        lr=1e-4,
        l2_strength=1e-4
    ):
        super().__init__()
        self.save_hyperparameters()

        # 단순 Linear Head 예시
        self.fc = nn.Linear(input_dim, n_cls)
        
        # Metric 정의
        self.train_acc = Accuracy(task="binary" if n_cls == 2 else "multiclass", num_classes=n_cls)
        self.val_acc   = Accuracy(task="binary" if n_cls == 2 else "multiclass", num_classes=n_cls)
        self.test_acc  = Accuracy(task="binary" if n_cls == 2 else "multiclass", num_classes=n_cls)

        self.train_auc = AUROC(task="binary" if n_cls == 2 else "multiclass", num_classes=n_cls)
        self.val_auc   = AUROC(task="binary" if n_cls == 2 else "multiclass", num_classes=n_cls)
        self.test_auc  = AUROC(task="binary" if n_cls == 2 else "multiclass", num_classes=n_cls)

    def forward(self, x):
        # x: (B, D)
        return self.fc(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y)
        self.train_auc.update(logits, y)
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        acc = self.train_acc.compute()
        auc = self.train_auc.compute()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_auc", auc, prog_bar=True)
        self.train_acc.reset()
        self.train_auc.reset()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        self.val_acc.update(preds, y)
        self.val_auc.update(logits, y)
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def on_validation_epoch_end(self):
        acc = self.val_acc.compute()
        auc = self.val_auc.compute()
        self.log("valid_acc", acc, prog_bar=True)
        self.log("valid_auc", auc, prog_bar=True)
        self.val_acc.reset()
        self.val_auc.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y)
        self.test_auc.update(logits, y)
        return loss

    def on_test_epoch_end(self):
        acc = self.test_acc.compute()
        auc = self.test_auc.compute()
        self.log("test_acc", acc, prog_bar=True)
        self.log("test_auc", auc, prog_bar=True)
        self.test_acc.reset()
        self.test_auc.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.hparams.lr, 
            weight_decay=self.hparams.l2_strength
        )
        return optimizer


In [15]:
feature_dir = 'feature/yeo_strat_kfold/'
x_data = np.load(os.path.join(feature_dir, "opensmile_feature.npy"))
print(x_data.shape)  # 예) (500, 988) ?

(32, 1, 988)


In [16]:
x_data = np.load(os.path.join(feature_dir, "clap_feature.npy"))
print(x_data.shape)  # 예) (500, 988) ?

(32, 1024)


In [30]:
import os
import numpy as np
import torch
import pytorch_lightning as pl
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader

def kfold_mydataset_with_baseline(
    feature_dir="feature/yeo_strat_kfold/",
    pretrain="operaCT",
    n_cls=2, 
    l2_strength=1e-4, 
    epochs=64, 
    batch_size=512, 
    lr=1e-4, 
    head="linear", 
    feat_dim=768,
    n_splits=5,            
    val_size=0.2,         
    random_state=42,
    input_sec=8
):
    print("*" * 48)
    print(f"[kfold_mydataset] n_splits={n_splits}, pretrain={pretrain}, head={head}, input_sec={input_sec}")
    print(f"l2_strength={l2_strength}, lr={lr}, feat_dim={feat_dim}, epochs={epochs}")

    # 시드 고정
    pl.seed_everything(random_state, workers=True)
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed(random_state)

    # 필요한 파일 로드
    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
    labels = np.load(os.path.join(feature_dir, "labels.npy"))
    patient_ids = np.load(os.path.join(feature_dir, "patient_ids.npy"))

    # (A) baseline 임베딩 여부 체크
    baseline_mode = pretrain in ["opensmile", "vggish", "clap", "audiomae"]

    # --------------------------------------------------------------------------------
    # 2) X_data 로드: baseline 임베딩 vs. 기존 스펙트로그램(or AudioMAE fbank 등)
    # --------------------------------------------------------------------------------
    if baseline_mode:
        # 예) opensmile_feature.npy, vggish_feature.npy, clap_feature.npy, audiomae_feature.npy
        baseline_feat_npy = os.path.join(feature_dir, f"{pretrain}_feature.npy")
        print(f"[kfold_mydataset] Baseline mode: loading {baseline_feat_npy}")
        if not os.path.exists(baseline_feat_npy):
            raise FileNotFoundError(
                f"{baseline_feat_npy} not found. 먼저 extract_and_save_embeddings_baselines()를 실행하세요."
            )
        x_data = np.load(baseline_feat_npy)
        print(f"x_data.shape={x_data.shape}, labels.shape={labels.shape}")
        #if x_data.ndim == 3 and x_data.shape[1] == 1:
        #    x_data = x_data.squeeze(1)  # (N, 988)

    else:
        # (기존) spectrogram or audioMAE 전용 fbank 등 로드
        if pretrain == "audiomae":
            # AudioMAE fbank 예시
            fbank_npy = os.path.join(feature_dir, "fbank_audiomae.npy")
            if not os.path.exists(fbank_npy):
                # 필요시 생성 로직
                # from src.util import get_split_signal_fbank_pad
                print(f"{fbank_npy} not found. 생성 혹은 준비가 필요합니다.")
                # (생성 예시) ...
            x_data = np.load(fbank_npy)
            print(f"AudioMAE fbank shape={x_data.shape}")
        else:
            # 일반 spectrogram 예시
            spec_npy = os.path.join(feature_dir, f"spectrogram_pad{int(input_sec)}.npy")
            if not os.path.exists(spec_npy):
                from src.util import get_split_signal_librosa
                print(f"[kfold_mydataset] {spec_npy} 파일이 없어 새로 생성합니다...")
                x_data_list = []
                for audio_file in sound_dir_loc:
                    spec_data = get_split_signal_librosa(
                        "",
                        audio_file[:-4],  # ".wav" 제거
                        spectrogram=True, 
                        input_sec=input_sec,
                        trim_tail=False
                    )[0]
                    x_data_list.append(spec_data)
                x_data = np.array(x_data_list)
                np.save(spec_npy, x_data)
                print(f"[kfold_mydataset] Saved {spec_npy}!")
            else:
                x_data = np.load(spec_npy)

        print(f"Total samples = {len(x_data)}, x_data.shape={x_data.shape}, labels.shape={labels.shape}")

    # 3) '환자별' 라벨 결정
    unique_pids = np.unique(patient_ids)
    pid_to_label = {}
    for pid in unique_pids:
        pid_labels = labels[patient_ids == pid]
        pid_to_label[pid] = int(np.any(pid_labels == 1))  # 하나라도 1이면 1

    pid_labels_array = np.array([pid_to_label[p] for p in unique_pids])

    # 4) k-Fold (환자 단위 StratifiedKFold)
    skf = StratifiedKFold(
        n_splits=n_splits, 
        shuffle=True, 
        random_state=random_state
    )

    fold_auc_list = []
    fold_acc_list = []

    for fold_idx, (trainval_index, test_index) in enumerate(skf.split(unique_pids, pid_labels_array)):
        print("\n" + "="*60)
        print(f"[Fold {fold_idx+1}/{n_splits}]")

        test_pids = unique_pids[test_index]
        trainval_pids = unique_pids[trainval_index]

        trainval_labels = np.array([pid_to_label[p] for p in trainval_pids])

        # (a) train/val 분할
        tr_pids, val_pids = train_test_split(
            trainval_pids,
            test_size=val_size,
            random_state=random_state,
            stratify=trainval_labels
        )

        # (b) 환자 -> index
        tr_idx = np.isin(patient_ids, tr_pids)
        val_idx = np.isin(patient_ids, val_pids)
        test_idx = np.isin(patient_ids, test_pids)

        x_train, y_train = x_data[tr_idx], labels[tr_idx]
        x_val,   y_val   = x_data[val_idx], labels[val_idx]
        x_test,  y_test  = x_data[test_idx], labels[test_idx]

        print(f"[Fold {fold_idx+1}] train={len(x_train)}, val={len(x_val)}, test={len(x_test)}")
        print(f"train_pids={tr_pids}, val_pids={val_pids}, test_pids={test_pids}")

        # (c) Dataset / DataLoader
        if baseline_mode:
            # Baseline 임베딩 전용 Dataset
            train_dataset = EmbeddingDataset(x_train, y_train)
            val_dataset   = EmbeddingDataset(x_val,   y_val)
            test_dataset  = EmbeddingDataset(x_test,  y_test)
        else:
            # 기존 AudioDataset (스펙트로그램 / fbank용)
            # AudioDataset의 구현에 맞게 인자 조정
            train_dataset = AudioDataset((x_train, y_train), augment=True,  max_len=False)
            val_dataset   = AudioDataset((x_val,   y_val),   augment=False, max_len=False)
            test_dataset  = AudioDataset((x_test,  y_test),  augment=False, max_len=False)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
        test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

        # (d) 모델 초기화
        if baseline_mode:
            # 임베딩 크기 자동 추정
            input_dim = x_train.shape[1]  # (N, D)에서 D
            model = EmbeddingClassifier(
                input_dim=input_dim,
                n_cls=n_cls,
                lr=lr,
                l2_strength=l2_strength
            )
        else:
            if pretrain == "audiomae":
                # AudioMAE용 예시
                from src.benchmark.baseline.audioMAE.models_mae import vit_base_patch16
                encoder_path = "src/benchmark/baseline/audioMAE/ViTB_pretrained.pth"
                ckpt = torch.load(encoder_path)

                net = vit_base_patch16(
                    in_chans=1,
                    img_size=(1024,128),  # 예시에 따라 다름
                    drop_path_rate=0.1,
                    global_pool=True,
                    mask_2d=False,
                    use_custom_patch=False
                )
                net.load_state_dict(ckpt["model"], strict=False)

                from your_code_path import AudioClassifierAudioMAE
                model = AudioClassifierAudioMAE(
                    net=net, 
                    head=head, 
                    classes=n_cls, 
                    lr=lr, 
                    l2_strength=l2_strength, 
                    feat_dim=feat_dim
                )

            elif pretrain == "clap":
                from src.benchmark.baseline.msclap import CLAP
                from your_code_path import AudioClassifierCLAP

                clap_model = CLAP(version='2022', use_cuda=True)
                net = clap_model.clap.audio_encoder

                model = AudioClassifierCLAP(
                    net=net, 
                    head=head, 
                    classes=n_cls, 
                    lr=lr, 
                    l2_strength=l2_strength, 
                    feat_dim=feat_dim
                )

            else:
                # 기존 operaCT, null 등
                pretrained_model = initialize_pretrained_model(pretrain)
                if pretrain != "null":
                    encoder_path = get_encoder_path(pretrain)
                    ckpt = torch.load(encoder_path, map_location="cpu")
                    pretrained_model.load_state_dict(ckpt["state_dict"], strict=False)

                net = pretrained_model.encoder
                from your_code_path import AudioClassifier
                model = AudioClassifier(
                    net=net, 
                    head=head, 
                    classes=n_cls, 
                    lr=lr, 
                    l2_strength=l2_strength, 
                    feat_dim=feat_dim,
                    freeze_encoder="none"
                )

        # (e) Trainer 설정
        checkpoint_callback = ModelCheckpoint(
            monitor="valid_auc",
            mode="max",
            dirpath="cks/finetune/kfold_results/",
            filename=f"kfold_{fold_idx+1}-{head}_{pretrain}_bs{batch_size}_lr{lr}_ep{epochs}_l2{l2_strength}" + "-{epoch:02d}-{valid_auc:.2f}",
            every_n_epochs=5,
        )

        trainer = pl.Trainer(
            max_epochs=epochs,
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            devices=1,
            logger=False,
            callbacks=[DecayLearningRate(), checkpoint_callback],
            gradient_clip_val=1.0,
            log_every_n_steps=1,
            enable_progress_bar=True
        )

        # (f) 학습
        trainer.fit(model, train_loader, val_loader)

        # (g) 테스트
        test_result = trainer.test(model, test_loader)
        if len(test_result) > 0:
            fold_auc = test_result[0].get("test_auc", np.nan)
            fold_acc = test_result[0].get("test_acc", np.nan)

            print(f"[Fold {fold_idx+1}] Test AUC={fold_auc:.4f}, ACC={fold_acc:.4f}")
            fold_auc_list.append(fold_auc)
            fold_acc_list.append(fold_acc)
        else:
            fold_auc_list.append(np.nan)
            fold_acc_list.append(np.nan)

    # 5) k-Fold 결과 요약
    fold_auc_list = np.array(fold_auc_list, dtype=np.float32)
    fold_acc_list = np.array(fold_acc_list, dtype=np.float32)

    valid_auc = fold_auc_list[~np.isnan(fold_auc_list)]
    valid_acc = fold_acc_list[~np.isnan(fold_acc_list)]

    mean_auc = np.mean(valid_auc) if len(valid_auc) > 0 else 0.0
    std_auc  = np.std(valid_auc)  if len(valid_auc) > 1 else 0.0
    mean_acc = np.mean(valid_acc) if len(valid_acc) > 0 else 0.0
    std_acc  = np.std(valid_acc)  if len(valid_acc) > 1 else 0.0

    print("="*60)
    print(f"Fold AUCs: {fold_auc_list}")
    print(f"Fold ACCs: {fold_acc_list}")
    print(f"Mean AUC: {mean_auc:.4f} ± {std_auc:.4f}")
    print(f"Mean ACC: {mean_acc:.4f} ± {std_acc:.4f}")
    print("="*60)

    return (mean_auc, std_auc, fold_auc_list), (mean_acc, std_acc, fold_acc_list)


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy, AUROC

class SimpleCNNClassifier(pl.LightningModule):
    def __init__(
        self, 
        n_cls=2, 
        lr=1e-4, 
        l2_strength=1e-4
    ):
        """
        - (B, 1, freq, time) 형태의 스펙트로그램을 입력
        - 2개의 Conv -> AdaptiveAvgPool2d -> Linear 구조
        - 이진분류(task="binary")를 위해 TorchMetrics 사용 시, 양성클래스 확률만 업데이트해야 함
        """
        super().__init__()
        self.save_hyperparameters()

        # 예시: 2번의 Conv(stride=2) -> 대략 크기 1/4로 축소 -> AdaptiveAvgPool2d
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size=(1, 1)),  # (B,32,1,1)
        )
        self.fc = nn.Linear(32, n_cls)  # 채널=32 => n_cls=2

        # 이진분류(binary)라고 가정
        self.train_acc = Accuracy(task="binary")
        self.valid_acc = Accuracy(task="binary")
        self.test_acc  = Accuracy(task="binary")

        self.train_auc = AUROC(task="binary")
        self.valid_auc = AUROC(task="binary")
        self.test_auc  = AUROC(task="binary")

    def forward(self, x):
        feat = self.conv_block(x)            # (B, 32, 1, 1)
        feat = feat.view(feat.size(0), -1)   # (B, 32)
        logits = self.fc(feat)               # (B, 2)
        return logits

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.l2_strength
        )

    def _compute_loss_and_metrics(self, batch, stage="train"):
        """
        - 이진분류: logits.shape=(B,2), target.shape=(B,)
        - TorchMetrics(task="binary")는 (B,) 형태의 확률만을 원함 -> pos_probs = probs[:,1]
        """
        x, y = batch
        logits = self.forward(x)           # (B, 2)
        loss = F.cross_entropy(logits, y)  # CrossEntropyLoss -> y in {0,1}

        probs = F.softmax(logits, dim=1)   # (B, 2)
        pos_probs = probs[:, 1]            # (B,) => 양성(1) 클래스 확률만 추출

        # stage별로 metric 업데이트
        if stage == "train":
            self.train_acc.update(pos_probs, y)
            self.train_auc.update(pos_probs, y)
        elif stage == "val":
            self.valid_acc.update(pos_probs, y)
            self.valid_auc.update(pos_probs, y)
        else:  # "test"
            self.test_acc.update(pos_probs, y)
            self.test_auc.update(pos_probs, y)

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._compute_loss_and_metrics(batch, stage="train")
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._compute_loss_and_metrics(batch, stage="val")
        self.log("val_loss", loss, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._compute_loss_and_metrics(batch, stage="test")
        self.log("test_loss", loss, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        acc = self.train_acc.compute()
        auc = self.train_auc.compute()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_auc", auc, prog_bar=True)
        self.train_acc.reset()
        self.train_auc.reset()

    def on_validation_epoch_end(self):
        acc = self.valid_acc.compute()
        auc = self.valid_auc.compute()
        self.log("valid_acc", acc, prog_bar=True)
        self.log("valid_auc", auc, prog_bar=True)
        self.valid_acc.reset()
        self.valid_auc.reset()

    def on_test_epoch_end(self):
        acc = self.test_acc.compute()
        auc = self.test_auc.compute()
        self.log("test_acc", acc, prog_bar=True)
        self.log("test_auc", auc, prog_bar=True)
        self.test_acc.reset()
        self.test_auc.reset()



In [23]:
import os
import numpy as np
import torch
import pytorch_lightning as pl
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader

# Lightning Callback 예: LearningRate Scheduler, Checkpoint 등
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

# ----------------------------
# (예시) AudioDataset 가정
# ----------------------------
# 실제로는 사용자 정의된 Dataset 코드가 필요합니다.
# 여기서는 (x_data, labels) tuple을 받아
# (tensor, label) 형태로 반환한다고 가정합니다.
import os
import numpy as np
import torch
import pytorch_lightning as pl
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader

# (Dataset) 스펙트럼 npy -> Tensor 변환
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, data_tuple):
        x_data, y_data = data_tuple
        self.x_data = x_data
        self.y_data = y_data
        
        # (N, freq, time) -> (N,1,freq,time)
        if len(self.x_data.shape) == 3:
            self.x_data = self.x_data[:, np.newaxis, :, :]

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.long)


def kfold_simple_cnn(
    feature_dir="feature/yeo_strat_kfold/",
    n_cls=2, 
    l2_strength=1e-4, 
    epochs=64, 
    batch_size=512, 
    lr=1e-4,
    n_splits=5,         
    val_size=0.2,       
    random_state=42,
    input_sec=8
):
    """k-Fold 교차검증: SimpleCNN (이진분류)"""

    print("*"*48)
    print(f"[kfold_simple_cnn] n_splits={n_splits}, lr={lr}, input_sec={input_sec}")
    print(f"l2_strength={l2_strength}, epochs={epochs}, batch_size={batch_size}")

    pl.seed_everything(random_state, workers=True)

    # 1) 필요한 npy 파일 로드
    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
    labels = np.load(os.path.join(feature_dir, "labels.npy"))
    patient_ids = np.load(os.path.join(feature_dir, "patient_ids.npy"))

    spec_npy = os.path.join(feature_dir, f"spectrogram_pad{int(input_sec)}.npy")
    if not os.path.exists(spec_npy):
        # 필요 시 spectrogram 생성 로직
        from src.util import get_split_signal_librosa
        x_data_list = []
        for audio_file in sound_dir_loc:
            spec_data = get_split_signal_librosa(
                "",
                audio_file[:-4],
                spectrogram=True, 
                input_sec=input_sec,
                trim_tail=False
            )[0]
            x_data_list.append(spec_data)
        x_data = np.array(x_data_list)
        np.save(spec_npy, x_data)
    else:
        x_data = np.load(spec_npy)

    # 2) 환자별 라벨(0/1)
    unique_pids = np.unique(patient_ids)
    pid_to_label = {}
    for pid in unique_pids:
        if np.any(labels[patient_ids == pid] == 1):
            pid_to_label[pid] = 1
        else:
            pid_to_label[pid] = 0
    pid_labels_array = np.array([pid_to_label[p] for p in unique_pids])

    # 3) k-Fold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fold_auc_list, fold_acc_list = [], []

    for fold_idx, (trainval_index, test_index) in enumerate(skf.split(unique_pids, pid_labels_array)):
        # test 환자/ train+val 환자
        test_pids = unique_pids[test_index]
        trainval_pids = unique_pids[trainval_index]
        trainval_labels = np.array([pid_to_label[p] for p in trainval_pids])

        # train / val split
        tr_pids, val_pids = train_test_split(
            trainval_pids,
            test_size=val_size,
            random_state=random_state,
            stratify=trainval_labels
        )

        tr_idx = np.isin(patient_ids, tr_pids)
        val_idx = np.isin(patient_ids, val_pids)
        test_idx = np.isin(patient_ids, test_pids)

        x_train, y_train = x_data[tr_idx], labels[tr_idx]
        x_val,   y_val   = x_data[val_idx], labels[val_idx]
        x_test,  y_test  = x_data[test_idx], labels[test_idx]

        print("\n"+"="*60)
        print(f"[Fold {fold_idx+1}/{n_splits}] train={len(x_train)}, val={len(x_val)}, test={len(x_test)}")

        # DataLoader
        train_dataset = AudioDataset((x_train, y_train))
        val_dataset   = AudioDataset((x_val,   y_val))
        test_dataset  = AudioDataset((x_test,  y_test))

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
        test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

        # 4) 모델 초기화 (이진분류 + Adaptive Pool)
        model = SimpleCNNClassifier(
            n_cls=n_cls, 
            lr=lr,
            l2_strength=l2_strength
        )

        # 체크포인트 콜백
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            monitor="valid_auc",
            mode="max",
            dirpath="cks/simple_cnn_kfold/",
            filename=(
                f"simpleCNN_kfold_{fold_idx+1}_bs{batch_size}_lr{lr}_ep{epochs}_l2{l2_strength}"
                "-{epoch:02d}-{valid_auc:.3f}"
            ),
            every_n_epochs=5,
        )

        # Trainer
        trainer = pl.Trainer(
            max_epochs=epochs,
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            devices=1,
            logger=False,
            callbacks=[checkpoint_callback],
            gradient_clip_val=1.0,
            log_every_n_steps=1,
            enable_progress_bar=True
        )

        # 5) 학습
        trainer.fit(model, train_loader, val_loader)

        # 6) 테스트
        test_result = trainer.test(model, test_loader)
        if len(test_result) > 0:
            fold_auc = test_result[0].get("test_auc", np.nan)
            fold_acc = test_result[0].get("test_acc", np.nan)
            fold_auc_list.append(fold_auc)
            fold_acc_list.append(fold_acc)
            print(f"[Fold {fold_idx+1}] Test AUC={fold_auc:.4f}, ACC={fold_acc:.4f}")
        else:
            fold_auc_list.append(np.nan)
            fold_acc_list.append(np.nan)

    # 7) 결과 요약
    fold_auc_list = np.array(fold_auc_list, dtype=np.float32)
    fold_acc_list = np.array(fold_acc_list, dtype=np.float32)

    valid_auc = fold_auc_list[~np.isnan(fold_auc_list)]
    valid_acc = fold_acc_list[~np.isnan(fold_acc_list)]

    mean_auc = np.mean(valid_auc) if len(valid_auc) > 0 else 0.0
    std_auc  = np.std(valid_auc)  if len(valid_auc) > 1 else 0.0
    mean_acc = np.mean(valid_acc) if len(valid_acc) > 0 else 0.0
    std_acc  = np.std(valid_acc)  if len(valid_acc) > 1 else 0.0

    print("="*60)
    print(f"Fold AUCs: {fold_auc_list}")
    print(f"Fold ACCs: {fold_acc_list}")
    print(f"Mean AUC: {mean_auc:.4f} ± {std_auc:.4f}")
    print(f"Mean ACC: {mean_acc:.4f} ± {std_acc:.4f}")
    print("="*60)

    return (mean_auc, std_auc, fold_auc_list), (mean_acc, std_acc, fold_acc_list)


In [26]:
if __name__ == "__main__":
    feature_dir = "feature/yeo_strat_kfold/"

    # 1) Transfer Learning (기존 코드) 예시
    # mean_auc_info, mean_acc_info = kfold_mydataset(
    #     feature_dir=feature_dir,
    #     pretrain="operaCT",
    #     n_cls=2, 
    #     l2_strength=1e-4, 
    #     epochs=100,    
    #     batch_size=256,
    #     lr=1e-4, 
    #     head="linear", 
    #     feat_dim=768,
    #     n_splits=5,
    #     val_size=0.2,
    #     random_state=42,
    #     input_sec=60
    # )

    # 2) Simple CNN (No Pretrain) 예시
    mean_auc_info, mean_acc_info = kfold_simple_cnn(
        feature_dir=feature_dir,
        n_cls=2,
        l2_strength=1e-4,
        epochs=100,         
        batch_size=256,    
        lr=1e-4,
        n_splits=5,
        val_size=0.2,
        random_state=42,
        input_sec=8
    )

    (mean_auc, std_auc, fold_auc_arr) = mean_auc_info
    (mean_acc, std_acc, fold_acc_arr) = mean_acc_info

    print("[Final Result - SimpleCNN]")
    print("Fold AUCs:", fold_auc_arr)
    print(f"Mean AUC: {mean_auc:.3f} ± {std_auc:.3f}")
    print("Fold ACCs:", fold_acc_arr)
    print(f"Mean ACC: {mean_acc:.3f} ± {std_acc:.3f}")


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/audio/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /workspace/cks/simple_cnn_kfold exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

  | Name       | Type           | Params
----------------------------------------------
0 | conv_block | Sequential     | 4.8 K 
1 | fc         | Linear         | 66    
2 | train_acc  | BinaryAccuracy | 0     
3 | valid_acc  | BinaryAccuracy | 0     
4 | test_acc   | BinaryAccuracy | 0     
5 | train_auc  | BinaryAUROC    | 0     
6 | valid_auc  | BinaryAUROC    | 0     
7 | test_auc   | BinaryAUROC    | 0     
----------------------------------------------
4.9 K     Trainable params
0         Non-trainable params
4.9 K     Total params
0.019     Total estimated model params size (MB)


************************************************
[kfold_simple_cnn] n_splits=5, lr=0.0001, input_sec=8
l2_strength=0.0001, epochs=100, batch_size=256

[Fold 1/5] train=19, val=5, test=8


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.4285714328289032
        test_auc            0.4166666865348816
        test_loss           0.7351469397544861
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[Fold 5] Test AUC=0.4167, ACC=0.4286
Fold AUCs: [0.625     0.5       0.        0.375     0.4166667]
Fold ACCs: [0.5        0.2        0.6666667  0.6666667  0.42857143]
Mean AUC: 0.3833 ± 0.2098
Mean ACC: 0.4924 ± 0.1734
[Final Result - SimpleCNN]
Fold AUCs: [0.625     0.5       0.        0.375     0.4166667]
Mean AUC: 0.383 ± 0.210
Fold ACCs: [0.5        0.2        0.6666667  0.6666667  0.42857143]
Mean ACC: 0.492 ± 0.173


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy, AUROC

class TimeFreqSepModel(pl.LightningModule):
    """
    - 오디오 스펙트로그램(2D) 입력
    - 첫 번째 블록에서 '시간축'만을 컨볼루션
    - 두 번째 블록에서 '주파수축'만을 컨볼루션
    - 그 뒤 Adaptive Pooling으로 최종 Linear 분류
    """

    def __init__(
        self, 
        n_cls=2, 
        lr=1e-4, 
        l2_strength=1e-4,
        task="binary"  # 이진분류("binary") 혹은 멀티클래스("multiclass")
    ):
        super().__init__()
        self.save_hyperparameters()

        # -------------------------
        # 1) 첫 번째 컨볼루션: 시간축에만 1D 필터
        #    - (kernel_size=(1,5)) => freq 축은 1, time 축은 5
        #    - stride=(1,2) => time축을 2배 줄이기
        #    - padding=(0,2) => freq축에는 패딩X, time축에는 패딩2
        # -------------------------
        self.conv_time_1 = nn.Conv2d(
            in_channels=1, out_channels=8,
            kernel_size=(1,5), stride=(1,2), padding=(0,2)
        )
        self.bn_time_1 = nn.BatchNorm2d(8)

        # -------------------------
        # 2) 두 번째 컨볼루션: 주파수축에만 1D 필터
        #    - (kernel_size=(5,1)) => freq 축은 5, time 축은 1
        #    - stride=(2,1) => freq축을 2배 줄이기
        #    - padding=(2,0) => freq축 패딩2, time축 패딩X
        # -------------------------
        self.conv_freq_1 = nn.Conv2d(
            in_channels=8, out_channels=16,
            kernel_size=(5,1), stride=(2,1), padding=(2,0)
        )
        self.bn_freq_1 = nn.BatchNorm2d(16)

        # -------------------------
        # 3) 두 번째 블록(선택사항): 더 깊게 하고 싶다면 추가로 만들 수 있음
        #    여기서는 간단히 한 블록만 예시
        # -------------------------

        # Adaptive Avg Pooling => (B,16,1,1) 로 축소
        self.gap = nn.AdaptiveAvgPool2d(output_size=(1,1))

        # 최종 분류
        self.fc = nn.Linear(16, n_cls)

        # -------------------------
        # 4) Metric 정의
        # -------------------------
        if task == "binary":
            self.train_acc = Accuracy(task="binary")
            self.train_auc = AUROC(task="binary")
            self.valid_acc = Accuracy(task="binary")
            self.valid_auc = AUROC(task="binary")
            self.test_acc  = Accuracy(task="binary")
            self.test_auc  = AUROC(task="binary")
        else:
            self.train_acc = Accuracy(task="multiclass", num_classes=n_cls)
            self.train_auc = AUROC(task="multiclass", num_classes=n_cls)
            self.valid_acc = Accuracy(task="multiclass", num_classes=n_cls)
            self.valid_auc = AUROC(task="multiclass", num_classes=n_cls)
            self.test_acc  = Accuracy(task="multiclass", num_classes=n_cls)
            self.test_auc  = AUROC(task="multiclass", num_classes=n_cls)

        self.task = task
        self.n_cls = n_cls

    def forward(self, x):
        """
        x: (B, 1, freq, time)
        """
        # 시간축 컨볼루션
        x = F.relu(self.bn_time_1(self.conv_time_1(x)))   # => (B,8,freq,(time/2))
        # 주파수축 컨볼루션
        x = F.relu(self.bn_freq_1(self.conv_freq_1(x)))   # => (B,16,(freq/2),(time/2))

        # Pooling -> (B,16,1,1)
        x = self.gap(x)

        # Flatten -> (B,16)
        x = x.view(x.size(0), -1)

        # 최종 분류 -> (B,n_cls)
        logits = self.fc(x)
        return logits

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.l2_strength
        )

    def _compute_loss_and_metrics(self, batch, stage="train"):
        x, y = batch
        logits = self.forward(x)           # (B, n_cls)
        loss = F.cross_entropy(logits, y)  # 라벨 {0,1,...,n_cls-1}

        probs = F.softmax(logits, dim=1)   # (B, n_cls)

        # 이진분류인 경우, TorchMetrics(task="binary")는 (B,) 형태 확률을 기대
        if self.task == "binary" and self.n_cls == 2:
            pos_probs = probs[:, 1]  # 양성클래스 확률만 추출 (B,)
        else:
            # 멀티클래스인 경우 (B,n_cls) 그대로 사용
            pos_probs = probs

        if stage == "train":
            self.train_acc.update(pos_probs, y)
            self.train_auc.update(pos_probs, y)
        elif stage == "val":
            self.valid_acc.update(pos_probs, y)
            self.valid_auc.update(pos_probs, y)
        else:  # "test"
            self.test_acc.update(pos_probs, y)
            self.test_auc.update(pos_probs, y)

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._compute_loss_and_metrics(batch, stage="train")
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._compute_loss_and_metrics(batch, stage="val")
        self.log("val_loss", loss, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._compute_loss_and_metrics(batch, stage="test")
        self.log("test_loss", loss, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        acc = self.train_acc.compute()
        auc = self.train_auc.compute()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_auc", auc, prog_bar=True)
        self.train_acc.reset()
        self.train_auc.reset()

    def on_validation_epoch_end(self):
        acc = self.valid_acc.compute()
        auc = self.valid_auc.compute()
        self.log("valid_acc", acc, prog_bar=True)
        self.log("valid_auc", auc, prog_bar=True)
        self.valid_acc.reset()
        self.valid_auc.reset()

    def on_test_epoch_end(self):
        acc = self.test_acc.compute()
        auc = self.test_auc.compute()
        self.log("test_acc", acc, prog_bar=True)
        self.log("test_auc", auc, prog_bar=True)
        self.test_acc.reset()
        self.test_auc.reset()


In [28]:
def kfold_timefreq_model(
    feature_dir="feature/yeo_strat_kfold/",
    n_cls=2, 
    l2_strength=1e-4, 
    epochs=64, 
    batch_size=512, 
    lr=1e-4,
    n_splits=5,         
    val_size=0.2,       
    random_state=42,
    input_sec=8
):
    """Time-Frequency Separable CNN 기반 k-Fold 예시"""
    import os
    import numpy as np
    import torch
    import pytorch_lightning as pl
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from torch.utils.data import DataLoader

    pl.seed_everything(random_state, workers=True)

    # (1) 데이터 불러오기 (labels, patient_ids, spec_npy 등)
    sound_dir_loc = np.load(os.path.join(feature_dir, "sound_dir_loc.npy"))
    labels = np.load(os.path.join(feature_dir, "labels.npy"))
    patient_ids = np.load(os.path.join(feature_dir, "patient_ids.npy"))

    spec_npy = os.path.join(feature_dir, f"spectrogram_pad{int(input_sec)}.npy")
    if not os.path.exists(spec_npy):
        from src.util import get_split_signal_librosa
        x_data_list = []
        for audio_file in sound_dir_loc:
            spec_data = get_split_signal_librosa(
                "",
                audio_file[:-4],
                spectrogram=True, 
                input_sec=input_sec,
                trim_tail=False
            )[0]
            x_data_list.append(spec_data)
        x_data = np.array(x_data_list)
        np.save(spec_npy, x_data)
    else:
        x_data = np.load(spec_npy)

    # (2) 환자별 라벨
    unique_pids = np.unique(patient_ids)
    pid_to_label = {}
    for pid in unique_pids:
        if np.any(labels[patient_ids == pid] == 1):
            pid_to_label[pid] = 1
        else:
            pid_to_label[pid] = 0
    pid_labels_array = np.array([pid_to_label[p] for p in unique_pids])

    # (3) Dataset
    class AudioDataset(torch.utils.data.Dataset):
        def __init__(self, data_tuple):
            x_data, y_data = data_tuple
            if len(x_data.shape) == 3:
                x_data = x_data[:, np.newaxis, :, :]
            self.x_data = x_data
            self.y_data = y_data

        def __len__(self):
            return len(self.x_data)

        def __getitem__(self, idx):
            x = torch.tensor(self.x_data[idx], dtype=torch.float)
            y = torch.tensor(self.y_data[idx], dtype=torch.long)
            return x, y

    # (4) K-Fold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fold_auc_list = []
    fold_acc_list = []

    for fold_idx, (trainval_index, test_index) in enumerate(skf.split(unique_pids, pid_labels_array)):
        test_pids = unique_pids[test_index]
        trainval_pids = unique_pids[trainval_index]
        trainval_labels = np.array([pid_to_label[p] for p in trainval_pids])

        tr_pids, val_pids = train_test_split(
            trainval_pids,
            test_size=val_size,
            random_state=random_state,
            stratify=trainval_labels
        )

        tr_idx = np.isin(patient_ids, tr_pids)
        val_idx = np.isin(patient_ids, val_pids)
        test_idx = np.isin(patient_ids, test_pids)

        x_train, y_train = x_data[tr_idx], labels[tr_idx]
        x_val,   y_val   = x_data[val_idx], labels[val_idx]
        x_test,  y_test  = x_data[test_idx], labels[test_idx]

        train_dataset = AudioDataset((x_train, y_train))
        val_dataset   = AudioDataset((x_val,   y_val))
        test_dataset  = AudioDataset((x_test,  y_test))

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
        test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

        print(f"\nFold {fold_idx+1}/{n_splits} => train={len(train_dataset)}, val={len(val_dataset)}, test={len(test_dataset)}")

        # (5) 'TimeFreqSepModel' 사용
        model = TimeFreqSepModel(
            n_cls=n_cls,
            lr=lr,
            l2_strength=l2_strength,
            task="binary"  # 이진분류
        )

        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            monitor="valid_auc",
            mode="max",
            dirpath="cks/timefreq_kfold/",
            filename=(
                f"timefreq_kfold_{fold_idx+1}_bs{batch_size}_lr{lr}_ep{epochs}_l2{l2_strength}"
                "-{epoch:02d}-{valid_auc:.3f}"
            ),
            every_n_epochs=5,
        )

        trainer = pl.Trainer(
            max_epochs=epochs,
            accelerator="gpu" if torch.cuda.is_available() else "cpu",
            devices=1,
            logger=False,
            callbacks=[checkpoint_callback],
            enable_progress_bar=True
        )

        trainer.fit(model, train_loader, val_loader)
        test_result = trainer.test(model, test_loader)
        if len(test_result) > 0:
            fold_auc = test_result[0].get("test_auc", float('nan'))
            fold_acc = test_result[0].get("test_acc", float('nan'))
            fold_auc_list.append(fold_auc)
            fold_acc_list.append(fold_acc)
            print(f"[Fold {fold_idx+1}] Test AUC={fold_auc:.4f}, ACC={fold_acc:.4f}")

    # (6) 결과 요약
    fold_auc_list = np.array(fold_auc_list, dtype=np.float32)
    fold_acc_list = np.array(fold_acc_list, dtype=np.float32)

    valid_auc = fold_auc_list[~np.isnan(fold_auc_list)]
    valid_acc = fold_acc_list[~np.isnan(fold_acc_list)]

    mean_auc = np.mean(valid_auc) if len(valid_auc) > 0 else 0.0
    std_auc  = np.std(valid_auc)  if len(valid_auc) > 1 else 0.0
    mean_acc = np.mean(valid_acc) if len(valid_acc) > 0 else 0.0
    std_acc  = np.std(valid_acc)  if len(valid_acc) > 1 else 0.0

    print("="*60)
    print(f"Fold AUCs: {fold_auc_list}")
    print(f"Fold ACCs: {fold_acc_list}")
    print(f"Mean AUC: {mean_auc:.4f} ± {std_auc:.4f}")
    print(f"Mean ACC: {mean_acc:.4f} ± {std_acc:.4f}")
    print("="*60)

    return (mean_auc, std_auc, fold_auc_list), (mean_acc, std_acc, fold_acc_list)


In [29]:
if __name__ == "__main__":
    feature_dir = "feature/yeo_strat_kfold/"

    # 1) Transfer Learning (기존 코드) 예시
    # mean_auc_info, mean_acc_info = kfold_mydataset(
    #     feature_dir=feature_dir,
    #     pretrain="operaCT",
    #     n_cls=2, 
    #     l2_strength=1e-4, 
    #     epochs=100,    
    #     batch_size=256,
    #     lr=1e-4, 
    #     head="linear", 
    #     feat_dim=768,
    #     n_splits=5,
    #     val_size=0.2,
    #     random_state=42,
    #     input_sec=60
    # )

    # 2) Simple CNN (No Pretrain) 예시
    mean_auc_info, mean_acc_info = kfold_timefreq_model(
        feature_dir=feature_dir,
        n_cls=2,
        l2_strength=1e-4,
        epochs=100,         
        batch_size=256,    
        lr=1e-4,
        n_splits=5,
        val_size=0.2,
        random_state=42,
        input_sec=8
    )

    (mean_auc, std_auc, fold_auc_arr) = mean_auc_info
    (mean_acc, std_acc, fold_acc_arr) = mean_acc_info

    print("[Final Result - TF CNN]")
    print("Fold AUCs:", fold_auc_arr)
    print(f"Mean AUC: {mean_auc:.3f} ± {std_auc:.3f}")
    print("Fold ACCs:", fold_acc_arr)
    print(f"Mean ACC: {mean_acc:.3f} ± {std_acc:.3f}")


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]

   | Name        | Type              | Params
---------------------------------------------------
0  | conv_time_1 | Conv2d            | 48    
1  | bn_time_1   | BatchNorm2d       | 16    
2  | conv_freq_1 | Conv2d            | 656   
3  | bn_freq_1   | BatchNorm2d       | 32    
4  | gap         | AdaptiveAvgPool2d | 0     
5  | fc          | Linear            | 34    
6  | train_acc   | BinaryAccuracy    | 0     
7  | train_auc   | BinaryAUROC       | 0     
8  | valid_acc   | BinaryAccuracy    | 0     
9  | valid_auc   | BinaryAUROC       | 0     
10 | test_acc    | BinaryAccuracy    | 0     
11 | test_auc    | BinaryAUROC       | 0     
---------------------------------------------------
786       Trainable params
0         Non-trainable params
786       Total pa


Fold 1/5 => train=19, val=5, test=8


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6]


Testing: |                                                | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.4285714328289032
        test_auc            0.4166666865348816
        test_loss           0.7541521787643433
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[Fold 5] Test AUC=0.4167, ACC=0.4286
Fold AUCs: [0.6875    0.75      1.        0.75      0.4166667]
Fold ACCs: [0.5        0.6        0.6666667  0.6666667  0.42857143]
Mean AUC: 0.7208 ± 0.1862
Mean ACC: 0.5724 ± 0.0943
[Final Result - TF CNN]
Fold AUCs: [0.6875    0.75      1.        0.75      0.4166667]
Mean AUC: 0.721 ± 0.186
Fold ACCs: [0.5        0.6        0.6666667  0.6666667  0.42857143]
Mean ACC: 0.572 ± 0.094
